In [27]:
spark.stop()

In [1]:
import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
import pyspark.sql.functions as F
from pyspark.sql.types import *
import time
from pyrecdp.data_processor import *
from pyrecdp.encoder import *
from pyrecdp.utils import *

spark = SparkSession.builder.master('yarn')\
        .appName("Recsys2021_data_process")\
        .getOrCreate()
current_path = "/recsys2021_0608_processed/sample_0_3_16days_eric_features/"
path_prefix = "hdfs://"
dicts_folder = "recsys_dicts/"

In [ ]:
train = spark.read.parquet("")

In [13]:
train_lookup_path = "/recsys2021_0608_processed/sample_0_3_16days/train_lookup"
lookup_table_list = [
    "lookup_table_original_engaged_with_user_id",
    "lookup_table_original_engaged_with_user_id_original_tweet_type_original_language",
    "lookup_table_original_engaging_user_id",
    "lookup_table_original_hashtags_original_present_media_original_tweet_type_original_language",
    #"lookup_table_original_language",
    "lookup_table_original_present_domains_original_language_engagee_follows_engager_original_tweet_type_original_present_media_engaged_with_user_is_verified",
    "lookup_table_original_present_domains_original_present_media_original_tweet_type_original_language",
    "lookup_table_original_present_links_original_present_media_original_tweet_type_original_language",
    #"lookup_table_original_present_media",
    "lookup_table_original_present_media_original_tweet_type_original_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager",
    #"lookup_table_original_tweet_type",
    #"lookup_table_text_tokens",
    "lookup_table_text_tokens_original_tweet_type_original_language",
    #lookup_table_tw_first_word_original_tweet_type_original_language",
    #"lookup_table_tw_last_word_original_tweet_type_original_language",
    #"lookup_table_word_tweet_type_language",
]
lookup_eric_list = [
    "CE_hashtags_like_timestamp_indicator_fold.parquet",
    "CE_hashtags_reply_timestamp_indicator_fold.parquet",
    "CE_hashtags_retweet_timestamp_indicator_fold.parquet",
    "CE_hashtags_retweet_with_comment_timestamp_indicator_fold.parquet",
    "CE_mentions_like_timestamp_indicator_fold.parquet",
    "CE_mentions_reply_timestamp_indicator_fold.parquet",
    "CE_mentions_retweet_timestamp_indicator_fold.parquet",
    "CE_mentions_retweet_with_comment_timestamp_indicator_fold.parquet",
    "CE_present_domains_like_timestamp_indicator_fold.parquet",
    "CE_present_domains_reply_timestamp_indicator_fold.parquet",
    "CE_present_domains_retweet_timestamp_indicator_fold.parquet",
    "CE_present_domains_retweet_with_comment_timestamp_indicator_fold.parquet",
    "CPD_hashtags_engaging_user_id_fold.parquet",
    "CPD_mentions_engaging_user_id_fold.parquet",
    "CPD_present_domains_engaging_user_id_fold.parquet",
    "GMTE_engaged_with_user_id_engaging_user_id_fold.parquet",
    "GMTE_has_rt_engaged_with_user_id_fold.parquet",
    "GMTE_language_engaged_with_user_id_fold.parquet",
    "GMTE_language_engaging_user_id_fold.parquet",
    "GMTE_tweet_dow_engaged_with_user_id_fold.parquet",
    "GMTE_tweet_dow_engaging_user_id_fold.parquet",
    "GMTE_tweet_hour_engaged_with_user_id_fold.parquet",
    "GMTE_tweet_hour_engaging_user_id_fold.parquet",
    "GMTE_tweet_type_engaged_with_user_id_fold.parquet",
    "GMTE_tweet_type_engaging_user_id_fold.parquet",
    "TE_engaged_with_user_id_fold.parquet",
    "TE_engaging_user_id_fold.parquet",
    "TE_language_fold.parquet",
    "TE_tweet_dow_fold.parquet",
    "TE_tweet_hour_fold.parquet",
    "TE_tweet_type_fold.parquet",
]
eric_dfs = [(f, spark.read.parquet("file:////mnt/nvme2/chendi/BlueWhale/EricFeatures/%s" % (f))) for f in lookup_eric_list]
lookup_table_dfs = [(f, spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days/train_lookup/%s" % f)) for f in lookup_table_list]

In [16]:
after_merge_only_eric = {}
feature_list_after_merge = []
#for f, df in lookup_table_dfs:
#    input_cols = ['fold'] + [colname for colname in df.columns if ('TE' not in colname and 'CE' not in colname)]
#    target_cols = [colname for colname in df.columns if ('TE' in colname or 'CE' in colname)]
#    str_input_cols = str(set(input_cols))
#    if str_input_cols not in after_merge:
#        after_merge[str_input_cols] = {}
#    for out in target_cols:
#        after_merge[str_input_cols][out] = (input_cols, f, 0)
    
for f, df in eric_dfs:
    input_cols = [colname for colname in df.columns if ('indicator' not in colname and 'intersection' not in colname)]
    target_cols = [colname for colname in df.columns if ('indicator' in colname or 'intersection' in colname)]
    str_input_cols = str(set(input_cols))
    for out in target_cols:
        if (out in good_features_list):  
            feature_list_after_merge.append(out)
            if str_input_cols not in after_merge_only_eric:
                after_merge_only_eric[str_input_cols] = {}
            if f not in after_merge_only_eric[str_input_cols]:
                after_merge_only_eric[str_input_cols][f] = (input_cols, f, 0)
        
for in_col, te in after_merge_only_eric.items():
    #print(in_col)
    for k, v in te.items():
        print(v[0])
    #print(te)
    #print()
print(len(feature_list_after_merge))
for feature in feature_list_after_merge:
    print(feature)

['fold', 'engaging_user_id', 'engaged_with_user_id']
['fold', 'engaged_with_user_id', 'has_rt']
['fold', 'engaged_with_user_id', 'language']
['fold', 'engaging_user_id', 'language']
['fold', 'engaged_with_user_id', 'tweet_dow']
['fold', 'engaging_user_id', 'tweet_dow']
['fold', 'engaged_with_user_id', 'tweet_hour']
['fold', 'engaging_user_id', 'tweet_hour']
['fold', 'engaged_with_user_id', 'tweet_type']
['fold', 'engaging_user_id', 'tweet_type']
['fold', 'engaged_with_user_id']
['fold', 'engaging_user_id']
['fold', 'language']
['fold', 'tweet_dow']
['fold', 'tweet_type']
56
GTE_engaged_with_user_id_reply_timestamp_indicator_engaging_user_id
GTE_engaged_with_user_id_retweet_timestamp_indicator_engaging_user_id
GTE_engaged_with_user_id_retweet_with_comment_timestamp_indicator_engaging_user_id
GTE_engaged_with_user_id_like_timestamp_indicator_engaging_user_id
GTE_has_rt_reply_timestamp_indicator_engaged_with_user_id
GTE_has_rt_retweet_timestamp_indicator_engaged_with_user_id
GTE_has_rt_re

In [3]:
#print(after_merge_only_eric)
after_merge_only_eric = {"{'engaging_user_id', 'fold', 'engaged_with_user_id'}": {'GMTE_engaged_with_user_id_engaging_user_id_fold.parquet': (['fold', 'engaging_user_id', 'engaged_with_user_id'], 'GMTE_engaged_with_user_id_engaging_user_id_fold.parquet', 0)}, "{'fold', 'has_rt', 'engaged_with_user_id'}": {'GMTE_has_rt_engaged_with_user_id_fold.parquet': (['fold', 'engaged_with_user_id', 'has_rt'], 'GMTE_has_rt_engaged_with_user_id_fold.parquet', 0)}, "{'language', 'fold', 'engaged_with_user_id'}": {'GMTE_language_engaged_with_user_id_fold.parquet': (['fold', 'engaged_with_user_id', 'language'], 'GMTE_language_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'language'}": {'GMTE_language_engaging_user_id_fold.parquet': (['fold', 'engaging_user_id', 'language'], 'GMTE_language_engaging_user_id_fold.parquet', 0)}, "{'fold', 'tweet_dow', 'engaged_with_user_id'}": {'GMTE_tweet_dow_engaged_with_user_id_fold.parquet': (['fold', 'engaged_with_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'tweet_dow'}": {'GMTE_tweet_dow_engaging_user_id_fold.parquet': (['fold', 'engaging_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaging_user_id_fold.parquet', 0)}, "{'fold', 'tweet_hour', 'engaged_with_user_id'}": {'GMTE_tweet_hour_engaged_with_user_id_fold.parquet': (['fold', 'engaged_with_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'tweet_hour'}": {'GMTE_tweet_hour_engaging_user_id_fold.parquet': (['fold', 'engaging_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaging_user_id_fold.parquet', 0)}, "{'fold', 'tweet_type', 'engaged_with_user_id'}": {'GMTE_tweet_type_engaged_with_user_id_fold.parquet': (['fold', 'engaged_with_user_id', 'tweet_type'], 'GMTE_tweet_type_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'tweet_type'}": {'GMTE_tweet_type_engaging_user_id_fold.parquet': (['fold', 'engaging_user_id', 'tweet_type'], 'GMTE_tweet_type_engaging_user_id_fold.parquet', 0)}, "{'language', 'fold'}": {'TE_language_fold.parquet': (['fold', 'language'], 'TE_language_fold.parquet', 0)}, "{'fold', 'tweet_dow'}": {'TE_tweet_dow_fold.parquet': (['fold', 'tweet_dow'], 'TE_tweet_dow_fold.parquet', 0)}, "{'fold', 'tweet_type'}": {'TE_tweet_type_fold.parquet': (['fold', 'tweet_type'], 'TE_tweet_type_fold.parquet', 0)}}

In [49]:
after_merge={"{'fold', 'engaged_with_user_id'}": {'TE_engaged_with_user_id_reply_timestamp': (['fold', 'engaged_with_user_id', 'fold'], 'lookup_table_original_engaged_with_user_id', 0), 'TE_engaged_with_user_id_retweet_timestamp': (['fold', 'engaged_with_user_id', 'fold'], 'lookup_table_original_engaged_with_user_id', 0), 'TE_engaged_with_user_id_retweet_with_comment_timestamp': (['fold', 'engaged_with_user_id', 'fold'], 'lookup_table_original_engaged_with_user_id', 0), 'TE_engaged_with_user_id_like_timestamp': (['fold', 'engaged_with_user_id', 'fold'], 'lookup_table_original_engaged_with_user_id', 0), 'CE_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'fold'], 'lookup_table_original_engaged_with_user_id', 0), 'CE_engaged_with_user_id_norm': (['fold', 'engaged_with_user_id', 'fold'], 'lookup_table_original_engaged_with_user_id', 0)}, "{'language', 'fold', 'tweet_type', 'engaged_with_user_id'}": {'TE_engaged_with_user_id_tweet_type_language_reply_timestamp': (['fold', 'engaged_with_user_id', 'tweet_type', 'language', 'fold'], 'lookup_table_original_engaged_with_user_id_original_tweet_type_original_language', 0), 'TE_engaged_with_user_id_tweet_type_language_retweet_timestamp': (['fold', 'engaged_with_user_id', 'tweet_type', 'language', 'fold'], 'lookup_table_original_engaged_with_user_id_original_tweet_type_original_language', 0), 'TE_engaged_with_user_id_tweet_type_language_retweet_with_comment_timestamp': (['fold', 'engaged_with_user_id', 'tweet_type', 'language', 'fold'], 'lookup_table_original_engaged_with_user_id_original_tweet_type_original_language', 0), 'TE_engaged_with_user_id_tweet_type_language_like_timestamp': (['fold', 'engaged_with_user_id', 'tweet_type', 'language', 'fold'], 'lookup_table_original_engaged_with_user_id_original_tweet_type_original_language', 0)}, "{'engaging_user_id', 'fold'}": {'TE_engaging_user_id_reply_timestamp': (['fold', 'engaging_user_id', 'fold'], 'lookup_table_original_engaging_user_id', 0), 'TE_engaging_user_id_retweet_timestamp': (['fold', 'engaging_user_id', 'fold'], 'lookup_table_original_engaging_user_id', 0), 'TE_engaging_user_id_retweet_with_comment_timestamp': (['fold', 'engaging_user_id', 'fold'], 'lookup_table_original_engaging_user_id', 0), 'TE_engaging_user_id_like_timestamp': (['fold', 'engaging_user_id', 'fold'], 'lookup_table_original_engaging_user_id', 0), 'CE_engaging_user_id': (['fold', 'engaging_user_id', 'fold'], 'lookup_table_original_engaging_user_id', 0), 'CE_engaging_user_id_norm': (['fold', 'engaging_user_id', 'fold'], 'lookup_table_original_engaging_user_id', 0)}, "{'hashtags', 'tweet_type', 'present_media', 'language', 'fold'}": {'TE_hashtags_present_media_tweet_type_language_reply_timestamp': (['fold', 'hashtags', 'present_media', 'tweet_type', 'language', 'fold'], 'lookup_table_original_hashtags_original_present_media_original_tweet_type_original_language', 0), 'TE_hashtags_present_media_tweet_type_language_retweet_timestamp': (['fold', 'hashtags', 'present_media', 'tweet_type', 'language', 'fold'], 'lookup_table_original_hashtags_original_present_media_original_tweet_type_original_language', 0), 'TE_hashtags_present_media_tweet_type_language_retweet_with_comment_timestamp': (['fold', 'hashtags', 'present_media', 'tweet_type', 'language', 'fold'], 'lookup_table_original_hashtags_original_present_media_original_tweet_type_original_language', 0), 'TE_hashtags_present_media_tweet_type_language_like_timestamp': (['fold', 'hashtags', 'present_media', 'tweet_type', 'language', 'fold'], 'lookup_table_original_hashtags_original_present_media_original_tweet_type_original_language', 0)}, "{'tweet_type', 'present_media', 'engaged_with_user_is_verified', 'language', 'fold', 'engagee_follows_engager', 'present_domains'}": {'TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_reply_timestamp': (['fold', 'present_domains', 'language', 'engagee_follows_engager', 'tweet_type', 'present_media', 'engaged_with_user_is_verified', 'fold'], 'lookup_table_original_present_domains_original_language_engagee_follows_engager_original_tweet_type_original_present_media_engaged_with_user_is_verified', 0), 'TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_timestamp': (['fold', 'present_domains', 'language', 'engagee_follows_engager', 'tweet_type', 'present_media', 'engaged_with_user_is_verified', 'fold'], 'lookup_table_original_present_domains_original_language_engagee_follows_engager_original_tweet_type_original_present_media_engaged_with_user_is_verified', 0), 'TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_with_comment_timestamp': (['fold', 'present_domains', 'language', 'engagee_follows_engager', 'tweet_type', 'present_media', 'engaged_with_user_is_verified', 'fold'], 'lookup_table_original_present_domains_original_language_engagee_follows_engager_original_tweet_type_original_present_media_engaged_with_user_is_verified', 0), 'TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_like_timestamp': (['fold', 'present_domains', 'language', 'engagee_follows_engager', 'tweet_type', 'present_media', 'engaged_with_user_is_verified', 'fold'], 'lookup_table_original_present_domains_original_language_engagee_follows_engager_original_tweet_type_original_present_media_engaged_with_user_is_verified', 0)}, "{'tweet_type', 'present_media', 'language', 'fold', 'present_domains'}": {'TE_present_domains_present_media_tweet_type_language_reply_timestamp': (['fold', 'present_domains', 'present_media', 'tweet_type', 'language', 'fold'], 'lookup_table_original_present_domains_original_present_media_original_tweet_type_original_language', 0), 'TE_present_domains_present_media_tweet_type_language_like_timestamp': (['fold', 'present_domains', 'present_media', 'tweet_type', 'language', 'fold'], 'lookup_table_original_present_domains_original_present_media_original_tweet_type_original_language', 0)}, "{'present_links', 'tweet_type', 'present_media', 'language', 'fold'}": {'TE_present_links_present_media_tweet_type_language_reply_timestamp': (['fold', 'present_links', 'present_media', 'tweet_type', 'language', 'fold'], 'lookup_table_original_present_links_original_present_media_original_tweet_type_original_language', 0)}, "{'tweet_type', 'present_media', 'engaged_with_user_is_verified', 'language', 'fold', 'engagee_follows_engager', 'engaging_user_is_verified'}": {'TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_reply_timestamp': (['fold', 'present_media', 'tweet_type', 'language', 'engaged_with_user_is_verified', 'engaging_user_is_verified', 'engagee_follows_engager', 'fold'], 'lookup_table_original_present_media_original_tweet_type_original_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager', 0), 'TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_timestamp': (['fold', 'present_media', 'tweet_type', 'language', 'engaged_with_user_is_verified', 'engaging_user_is_verified', 'engagee_follows_engager', 'fold'], 'lookup_table_original_present_media_original_tweet_type_original_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager', 0), 'TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_with_comment_timestamp': (['fold', 'present_media', 'tweet_type', 'language', 'engaged_with_user_is_verified', 'engaging_user_is_verified', 'engagee_follows_engager', 'fold'], 'lookup_table_original_present_media_original_tweet_type_original_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager', 0), 'TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_like_timestamp': (['fold', 'present_media', 'tweet_type', 'language', 'engaged_with_user_is_verified', 'engaging_user_is_verified', 'engagee_follows_engager', 'fold'], 'lookup_table_original_present_media_original_tweet_type_original_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager', 0)}, "{'language', 'fold', 'tweet_type', 'text_tokens'}": {'TE_tw_hash0_tweet_type_language_reply_timestamp': (['fold', 'text_tokens', 'tweet_type', 'language', 'fold'], 'lookup_table_text_tokens_original_tweet_type_original_language', 0), 'TE_tw_hash0_tweet_type_language_retweet_timestamp': (['fold', 'text_tokens', 'tweet_type', 'language', 'fold'], 'lookup_table_text_tokens_original_tweet_type_original_language', 0), 'TE_tw_hash0_tweet_type_language_retweet_with_comment_timestamp': (['fold', 'text_tokens', 'tweet_type', 'language', 'fold'], 'lookup_table_text_tokens_original_tweet_type_original_language', 0), 'TE_tw_hash0_tweet_type_language_like_timestamp': (['fold', 'text_tokens', 'tweet_type', 'language', 'fold'], 'lookup_table_text_tokens_original_tweet_type_original_language', 0), 'TE_tw_hash1_tweet_type_language_retweet_with_comment_timestamp': (['fold', 'text_tokens', 'tweet_type', 'language', 'fold'], 'lookup_table_text_tokens_original_tweet_type_original_language', 0), 'TE_tw_uhash_tweet_type_language_retweet_with_comment_timestamp': (['fold', 'text_tokens', 'tweet_type', 'language', 'fold'], 'lookup_table_text_tokens_original_tweet_type_original_language', 0)}, "{'engaging_user_id', 'fold', 'engaged_with_user_id'}": {'GTE_engaged_with_user_id_reply_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'engaged_with_user_id'], 'GMTE_engaged_with_user_id_engaging_user_id_fold.parquet', 0), 'GTE_engaged_with_user_id_retweet_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'engaged_with_user_id'], 'GMTE_engaged_with_user_id_engaging_user_id_fold.parquet', 0), 'GTE_engaged_with_user_id_retweet_with_comment_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'engaged_with_user_id'], 'GMTE_engaged_with_user_id_engaging_user_id_fold.parquet', 0), 'GTE_engaged_with_user_id_like_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'engaged_with_user_id'], 'GMTE_engaged_with_user_id_engaging_user_id_fold.parquet', 0)}, "{'fold', 'has_rt', 'engaged_with_user_id'}": {'GTE_has_rt_reply_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'has_rt'], 'GMTE_has_rt_engaged_with_user_id_fold.parquet', 0), 'GTE_has_rt_retweet_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'has_rt'], 'GMTE_has_rt_engaged_with_user_id_fold.parquet', 0), 'GTE_has_rt_retweet_with_comment_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'has_rt'], 'GMTE_has_rt_engaged_with_user_id_fold.parquet', 0), 'GTE_has_rt_like_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'has_rt'], 'GMTE_has_rt_engaged_with_user_id_fold.parquet', 0)}, "{'language', 'fold', 'engaged_with_user_id'}": {'GTE_language_reply_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'language'], 'GMTE_language_engaged_with_user_id_fold.parquet', 0), 'GTE_language_retweet_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'language'], 'GMTE_language_engaged_with_user_id_fold.parquet', 0), 'GTE_language_retweet_with_comment_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'language'], 'GMTE_language_engaged_with_user_id_fold.parquet', 0), 'GTE_language_like_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'language'], 'GMTE_language_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'language'}": {'GTE_language_reply_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'language'], 'GMTE_language_engaging_user_id_fold.parquet', 0), 'GTE_language_retweet_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'language'], 'GMTE_language_engaging_user_id_fold.parquet', 0), 'GTE_language_retweet_with_comment_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'language'], 'GMTE_language_engaging_user_id_fold.parquet', 0), 'GTE_language_like_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'language'], 'GMTE_language_engaging_user_id_fold.parquet', 0)}, "{'fold', 'tweet_dow', 'engaged_with_user_id'}": {'GTE_tweet_dow_reply_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_dow_retweet_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_dow_like_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'tweet_dow'}": {'GTE_tweet_dow_reply_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaging_user_id_fold.parquet', 0), 'GTE_tweet_dow_retweet_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaging_user_id_fold.parquet', 0), 'GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaging_user_id_fold.parquet', 0), 'GTE_tweet_dow_like_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_dow'], 'GMTE_tweet_dow_engaging_user_id_fold.parquet', 0)}, "{'fold', 'tweet_hour', 'engaged_with_user_id'}": {'GTE_tweet_hour_reply_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_hour_retweet_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_hour_like_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'tweet_hour'}": {'GTE_tweet_hour_reply_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaging_user_id_fold.parquet', 0), 'GTE_tweet_hour_retweet_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaging_user_id_fold.parquet', 0), 'GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaging_user_id_fold.parquet', 0), 'GTE_tweet_hour_like_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_hour'], 'GMTE_tweet_hour_engaging_user_id_fold.parquet', 0)}, "{'fold', 'tweet_type', 'engaged_with_user_id'}": {'GTE_tweet_type_reply_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_type'], 'GMTE_tweet_type_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_type_retweet_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_type'], 'GMTE_tweet_type_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_type'], 'GMTE_tweet_type_engaged_with_user_id_fold.parquet', 0), 'GTE_tweet_type_like_timestamp_indicator_engaged_with_user_id': (['fold', 'engaged_with_user_id', 'tweet_type'], 'GMTE_tweet_type_engaged_with_user_id_fold.parquet', 0)}, "{'engaging_user_id', 'fold', 'tweet_type'}": {'GTE_tweet_type_reply_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_type'], 'GMTE_tweet_type_engaging_user_id_fold.parquet', 0), 'GTE_tweet_type_retweet_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_type'], 'GMTE_tweet_type_engaging_user_id_fold.parquet', 0), 'GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_type'], 'GMTE_tweet_type_engaging_user_id_fold.parquet', 0), 'GTE_tweet_type_like_timestamp_indicator_engaging_user_id': (['fold', 'engaging_user_id', 'tweet_type'], 'GMTE_tweet_type_engaging_user_id_fold.parquet', 0)}, "{'language', 'fold'}": {'TE_language_reply_timestamp_indicator': (['fold', 'language'], 'TE_language_fold.parquet', 0), 'TE_language_retweet_timestamp_indicator': (['fold', 'language'], 'TE_language_fold.parquet', 0), 'TE_language_retweet_with_comment_timestamp_indicator': (['fold', 'language'], 'TE_language_fold.parquet', 0), 'TE_language_like_timestamp_indicator': (['fold', 'language'], 'TE_language_fold.parquet', 0)}, "{'fold', 'tweet_dow'}": {'TE_tweet_dow_retweet_timestamp_indicator': (['fold', 'tweet_dow'], 'TE_tweet_dow_fold.parquet', 0)}, "{'fold', 'tweet_hour'}": {}, "{'fold', 'tweet_type'}": {'TE_tweet_type_reply_timestamp_indicator': (['fold', 'tweet_type'], 'TE_tweet_type_fold.parquet', 0), 'TE_tweet_type_retweet_timestamp_indicator': (['fold', 'tweet_type'], 'TE_tweet_type_fold.parquet', 0), 'TE_tweet_type_like_timestamp_indicator': (['fold', 'tweet_type'], 'TE_tweet_type_fold.parquet', 0)}}

In [ ]:
['fold', 
 'engaged_with_user_id', 
 'language', 
 'tweet_type', 
 'engaging_user_id', 
 'hashtags', 
 'present_media', 
 'engaged_with_user_is_verified', 
 'engagee_follows_engager', 
 'present_domains', 
 'present_links', 
 'engaging_user_is_verified', 
 'text_tokens', 
 'has_rt', 
 'tweet_dow', 
 'tweet_hour']

In [9]:
for in_col, te in after_merge_only_eric.items():
    #print(in_col)
    for k, v in te.items():
        print(v[0])
    #print(te)
    #print()

['fold', 'engaging_user_id', 'engaged_with_user_id']
['fold', 'engaged_with_user_id', 'has_rt']
['fold', 'engaged_with_user_id', 'language']
['fold', 'engaging_user_id', 'language']
['fold', 'engaged_with_user_id', 'tweet_dow']
['fold', 'engaging_user_id', 'tweet_dow']
['fold', 'engaged_with_user_id', 'tweet_hour']
['fold', 'engaging_user_id', 'tweet_hour']
['fold', 'engaged_with_user_id', 'tweet_type']
['fold', 'engaging_user_id', 'tweet_type']
['fold', 'language']
['fold', 'tweet_dow']
['fold', 'tweet_type']


In [ ]:
import json
import glob
default_col = {}
for filename in lookup_eric_list:
    json_filename = filename.replace("_fold.parquet", ".json")
    for name in glob.glob('/mnt/nvme2/chendi/BlueWhale/EricFeaturesSmall/%s/*.json' % json_filename):
        with open(name, 'r') as f:
            data = json.load(f)
            for k, v in data.items():                
                #print(v)
                v = eval(v)
                if "outputCols" not in v:
                    continue
                for k1 in v["outputCols"]:
                    default_col[k1] = v["mean"][k1]
                #print(def_v)
print(default_col)

In [40]:
default_col = {'hashtags_like_timestamp_indicator_intersection': 0.39640563540158563, 'hashtags_reply_timestamp_indicator_intersection': 0.027006638950719367, 'hashtags_retweet_timestamp_indicator_intersection': 0.08696896238375852, 'hashtags_retweet_with_comment_timestamp_indicator_intersection': 0.0067161671206384145, 'mentions_like_timestamp_indicator_intersection': 0.39640563540158563, 'mentions_reply_timestamp_indicator_intersection': 0.027006638950719367, 'mentions_retweet_timestamp_indicator_intersection': 0.08696896238375852, 'mentions_retweet_with_comment_timestamp_indicator_intersection': 0.0067161671206384145, 'present_domains_like_timestamp_indicator_intersection': 0.39640563540158563, 'present_domains_reply_timestamp_indicator_intersection': 0.027006638950719367, 'present_domains_retweet_timestamp_indicator_intersection': 0.08696896238375852, 'present_domains_retweet_with_comment_timestamp_indicator_intersection': 0.0067161671206384145, 'GTE_engaged_with_user_id_reply_timestamp_indicator_engaging_user_id': 0.027006638950719367, 'GTE_engaged_with_user_id_retweet_timestamp_indicator_engaging_user_id': 0.08696896238375852, 'GTE_engaged_with_user_id_retweet_with_comment_timestamp_indicator_engaging_user_id': 0.0067161671206384145, 'GTE_engaged_with_user_id_like_timestamp_indicator_engaging_user_id': 0.39640563540158563, 'GTE_has_rt_reply_timestamp_indicator_engaged_with_user_id': 0.027006638950719367, 'GTE_has_rt_retweet_timestamp_indicator_engaged_with_user_id': 0.08696896238375852, 'GTE_has_rt_retweet_with_comment_timestamp_indicator_engaged_with_user_id': 0.0067161671206384145, 'GTE_has_rt_like_timestamp_indicator_engaged_with_user_id': 0.39640563540158563, 'GTE_language_reply_timestamp_indicator_engaged_with_user_id': 0.027006638950719367, 'GTE_language_retweet_timestamp_indicator_engaged_with_user_id': 0.08696896238375852, 'GTE_language_retweet_with_comment_timestamp_indicator_engaged_with_user_id': 0.0067161671206384145, 'GTE_language_like_timestamp_indicator_engaged_with_user_id': 0.39640563540158563, 'GTE_language_reply_timestamp_indicator_engaging_user_id': 0.027006638950719367, 'GTE_language_retweet_timestamp_indicator_engaging_user_id': 0.08696896238375852, 'GTE_language_retweet_with_comment_timestamp_indicator_engaging_user_id': 0.0067161671206384145, 'GTE_language_like_timestamp_indicator_engaging_user_id': 0.39640563540158563, 'GTE_tweet_dow_reply_timestamp_indicator_engaged_with_user_id': 0.027006638950719367, 'GTE_tweet_dow_retweet_timestamp_indicator_engaged_with_user_id': 0.08696896238375852, 'GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaged_with_user_id': 0.0067161671206384145, 'GTE_tweet_dow_like_timestamp_indicator_engaged_with_user_id': 0.39640563540158563, 'GTE_tweet_dow_reply_timestamp_indicator_engaging_user_id': 0.027006638950719367, 'GTE_tweet_dow_retweet_timestamp_indicator_engaging_user_id': 0.08696896238375852, 'GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaging_user_id': 0.0067161671206384145, 'GTE_tweet_dow_like_timestamp_indicator_engaging_user_id': 0.39640563540158563, 'GTE_tweet_hour_reply_timestamp_indicator_engaged_with_user_id': 0.027006638950719367, 'GTE_tweet_hour_retweet_timestamp_indicator_engaged_with_user_id': 0.08696896238375852, 'GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaged_with_user_id': 0.0067161671206384145, 'GTE_tweet_hour_like_timestamp_indicator_engaged_with_user_id': 0.39640563540158563, 'GTE_tweet_hour_reply_timestamp_indicator_engaging_user_id': 0.027006638950719367, 'GTE_tweet_hour_retweet_timestamp_indicator_engaging_user_id': 0.08696896238375852, 'GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaging_user_id': 0.0067161671206384145, 'GTE_tweet_hour_like_timestamp_indicator_engaging_user_id': 0.39640563540158563, 'GTE_tweet_type_reply_timestamp_indicator_engaged_with_user_id': 0.027006638950719367, 'GTE_tweet_type_retweet_timestamp_indicator_engaged_with_user_id': 0.08696896238375852, 'GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaged_with_user_id': 0.0067161671206384145, 'GTE_tweet_type_like_timestamp_indicator_engaged_with_user_id': 0.39640563540158563, 'GTE_tweet_type_reply_timestamp_indicator_engaging_user_id': 0.027006638950719367, 'GTE_tweet_type_retweet_timestamp_indicator_engaging_user_id': 0.08696896238375852, 'GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaging_user_id': 0.0067161671206384145, 'GTE_tweet_type_like_timestamp_indicator_engaging_user_id': 0.39640563540158563, 'TE_engaged_with_user_id_reply_timestamp_indicator': 0.027006638950719367, 'TE_engaged_with_user_id_retweet_timestamp_indicator': 0.08696896238375852, 'TE_engaged_with_user_id_retweet_with_comment_timestamp_indicator': 0.0067161671206384145, 'TE_engaged_with_user_id_like_timestamp_indicator': 0.39640563540158563, 'TE_engaging_user_id_reply_timestamp_indicator': 0.027006638950719367, 'TE_engaging_user_id_retweet_timestamp_indicator': 0.08696896238375852, 'TE_engaging_user_id_retweet_with_comment_timestamp_indicator': 0.0067161671206384145, 'TE_engaging_user_id_like_timestamp_indicator': 0.39640563540158563, 'TE_language_reply_timestamp_indicator': 0.027006638950719367, 'TE_language_retweet_timestamp_indicator': 0.08696896238375852, 'TE_language_retweet_with_comment_timestamp_indicator': 0.0067161671206384145, 'TE_language_like_timestamp_indicator': 0.39640563540158563, 'TE_tweet_dow_reply_timestamp_indicator': 0.027006638950719367, 'TE_tweet_dow_retweet_timestamp_indicator': 0.08696896238375852, 'TE_tweet_dow_retweet_with_comment_timestamp_indicator': 0.0067161671206384145, 'TE_tweet_dow_like_timestamp_indicator': 0.39640563540158563, 'TE_tweet_hour_reply_timestamp_indicator': 0.027006638950719367, 'TE_tweet_hour_retweet_timestamp_indicator': 0.08696896238375852, 'TE_tweet_hour_retweet_with_comment_timestamp_indicator': 0.0067161671206384145, 'TE_tweet_hour_like_timestamp_indicator': 0.39640563540158563, 'TE_tweet_type_reply_timestamp_indicator': 0.027006638950719367, 'TE_tweet_type_retweet_timestamp_indicator': 0.08696896238375852, 'TE_tweet_type_retweet_with_comment_timestamp_indicator': 0.0067161671206384145, 'TE_tweet_type_like_timestamp_indicator': 0.39640563540158563}

In [41]:
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days/train_with_features")
train.printSchema()

root
 |-- tw_last_word: integer (nullable = true)
 |-- tw_first_word: integer (nullable = true)
 |-- text_tokens: string (nullable = true)
 |-- categorified_hashtags: integer (nullable = true)
 |-- categorified_tweet_id: integer (nullable = true)
 |-- categorified_present_media: long (nullable = true)
 |-- categorified_present_links: integer (nullable = true)
 |-- categorified_present_domains: integer (nullable = true)
 |-- categorified_tweet_type: long (nullable = true)
 |-- categorified_language: integer (nullable = true)
 |-- tweet_timestamp: integer (nullable = true)
 |-- categorified_engaged_with_user_id: integer (nullable = true)
 |-- engaged_with_user_follower_count: integer (nullable = true)
 |-- engaged_with_user_following_count: integer (nullable = true)
 |-- engaged_with_user_is_verified: boolean (nullable = true)
 |-- engaged_with_user_account_creation: integer (nullable = true)
 |-- categorified_engaging_user_id: integer (nullable = true)
 |-- engaging_user_follower_count:

In [42]:
current_path = "/recsys2021_0608_processed/sample_0_3_20days_eric_features"
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days/train_with_features")
df = train.withColumn('tweet_dow', F.col('dt_dow')).withColumn('tweet_hour', F.col('dt_hour'))
#df.printSchema()
df = df.withColumn('has_rt', F.col('original_tweet').rlike("^RT\s+|\sRT\s+"))
df.write.format('parquet').mode('overwrite').save("%s/train_with_features_has_rt" % current_path)

In [43]:
def merge_indicator_by_input_cols(proc, df, input_dicts, output_list, skip_list=[]):
    begin = time.time()
    for origin_cols, te in input_dicts.items():
        if origin_cols in skip_list:
            continue
        start = time.time()
        input_cols = list(eval(origin_cols))
        output_cols = []
        op_fillna_list = []

        for filename, i_dict in te.items():
            for out_col in i_dict[1].columns:
                if out_col not in output_cols and out_col not in input_cols:
                    output_cols.append(out_col)
                    default_v = default_col[out_col] if (out_col in default_col) else 0
                    op_fillna_list.append(FillNA([out_col], default_v))
        to_select = input_cols
        
        op_merge = ModelMerge(te)
        op_select = SelectFeature(input_cols + output_cols)
        distinct = Distinct()
        
        proc.reset_ops([op_merge] + [op_select, distinct] + op_fillna_list)
        output_list.append(proc.transform(df.select(*to_select), name="lookup_table_%s" % ('_'.join(input_cols))))
        print("Train Merge models for %s took: %.3f secs" % ("lookup_table_%s" % ('_'.join(input_cols)), time.time()-start))

############## Train join #############
after_merge_only_eric_dfs = {}
current_path = "/recsys2021_0608_processed/sample_0_3_20days_eric_features"
lookup_table_path = "file:///mnt/nvme2/chendi/BlueWhale/EricFeatures"
for input_cols, te in after_merge_only_eric.items():
    after_merge_only_eric_dfs[input_cols] = {}
    for out_cols, i_dict in te.items():
        after_merge_only_eric_dfs[input_cols][out_cols] = (i_dict[0], spark.read.parquet("%s/%s" % (lookup_table_path, i_dict[1])), i_dict[2])
    
proc = DataProcessor(spark, path_prefix,
                     current_path="%s/train_lookup/" % current_path, dicts_path=dicts_folder)
train_out_dfs = []
train = spark.read.parquet("%s/train_with_features_has_rt" % current_path)
merge_indicator_by_input_cols(proc, train, after_merge_only_eric_dfs, train_out_dfs)


Train Merge models for lookup_table_engaged_with_user_id_fold_engaging_user_id took: 71.420 secs
Train Merge models for lookup_table_engaged_with_user_id_fold_has_rt took: 18.815 secs
Train Merge models for lookup_table_engaged_with_user_id_fold_language took: 20.839 secs
Train Merge models for lookup_table_fold_language_engaging_user_id took: 27.233 secs
Train Merge models for lookup_table_engaged_with_user_id_fold_tweet_dow took: 23.742 secs
Train Merge models for lookup_table_fold_tweet_dow_engaging_user_id took: 31.603 secs
Train Merge models for lookup_table_engaged_with_user_id_fold_tweet_hour took: 25.633 secs
Train Merge models for lookup_table_fold_engaging_user_id_tweet_hour took: 37.807 secs
Train Merge models for lookup_table_engaged_with_user_id_fold_tweet_type took: 19.451 secs
Train Merge models for lookup_table_fold_tweet_type_engaging_user_id took: 28.915 secs
Train Merge models for lookup_table_engaged_with_user_id_fold took: 17.424 secs
Train Merge models for lookup_

In [45]:
# run it!
# This is used to generate valid_lookup from train_lookup
import pyspark.sql.functions as f
import pyspark.sql.functions as F
input_cols = [
    (['engaged_with_user_id'], "lookup_table_engaged_with_user_id_fold"),
    (['engaging_user_id'], "lookup_table_fold_engaging_user_id"),
    (['engaging_user_id', 'engaged_with_user_id'], "lookup_table_engaged_with_user_id_fold_engaging_user_id"),
    (['engaged_with_user_id', 'has_rt'], "lookup_table_engaged_with_user_id_fold_has_rt"),
    (['engaged_with_user_id', 'language'], "lookup_table_engaged_with_user_id_fold_language"),
    (['language'], "lookup_table_fold_language"),
    (['tweet_dow'], "lookup_table_fold_tweet_dow"),
    (['tweet_type'], "lookup_table_fold_tweet_type"),
    (['engaging_user_id', 'language'], "lookup_table_fold_language_engaging_user_id"),
    (['engaged_with_user_id', 'tweet_dow'], "lookup_table_engaged_with_user_id_fold_tweet_dow"),
    (['engaging_user_id', 'tweet_dow'], "lookup_table_fold_tweet_dow_engaging_user_id"),
    (['engaged_with_user_id', 'tweet_hour'], "lookup_table_engaged_with_user_id_fold_tweet_hour"),
    (['engaging_user_id', 'tweet_hour'], "lookup_table_fold_engaging_user_id_tweet_hour"),
    (['engaged_with_user_id', 'tweet_type'], "lookup_table_engaged_with_user_id_fold_tweet_type"),
    (['engaging_user_id', 'tweet_type'], "lookup_table_fold_tweet_type_engaging_user_id")
]

train_res_dfs = {}
current_path = "/recsys2021_0608_processed/sample_0_3_20days_eric_features"
for in_cols, dict_name in input_cols:
    start = time.time()
    df = spark.read.parquet("%s/train_lookup/%s" % (current_path, dict_name))
    encoding_indicators_df = df.drop('fold')
    for c in encoding_indicators_df.columns:
        if c in in_cols:
            encoding_indicators_df = encoding_indicators_df.drop(c)
    df = df.drop('fold').groupby(in_cols).agg(*[f.mean(c).alias(c) for c in encoding_indicators_df.columns])
    df.write.format("parquet").mode('overwrite').save("%s/valid_lookup/%s" % (current_path, dict_name))
    print("Generate valid_lookup for %s took %.3f secs" % (dict_name, (time.time() - start))) 


Generate valid_lookup for lookup_table_engaged_with_user_id_fold took 5.314 secs
Generate valid_lookup for lookup_table_fold_engaging_user_id took 5.646 secs
Generate valid_lookup for lookup_table_engaged_with_user_id_fold_engaging_user_id took 19.631 secs
Generate valid_lookup for lookup_table_engaged_with_user_id_fold_has_rt took 5.569 secs
Generate valid_lookup for lookup_table_engaged_with_user_id_fold_language took 5.917 secs
Generate valid_lookup for lookup_table_fold_language took 0.897 secs
Generate valid_lookup for lookup_table_fold_tweet_dow took 0.430 secs
Generate valid_lookup for lookup_table_fold_tweet_type took 0.336 secs
Generate valid_lookup for lookup_table_fold_language_engaging_user_id took 8.179 secs
Generate valid_lookup for lookup_table_engaged_with_user_id_fold_tweet_dow took 8.092 secs
Generate valid_lookup for lookup_table_fold_tweet_dow_engaging_user_id took 11.073 secs
Generate valid_lookup for lookup_table_engaged_with_user_id_fold_tweet_hour took 8.470 sec

In [3]:
current_path = "/recsys2021_0608_processed/sample_0_3_20days_eric_features"
train_feature_output_path = "%s" % (current_path)
valid_feature_output_path = "%s" % (current_path)

def merge_indicator_by_input_cols(proc, df, te, is_train=True, out_name="train_with_features"):
    start = time.time()
    op_merge = ModelMerge(te)
    proc.reset_ops([op_merge])
    proc.transform(df, name=out_name)
    if is_train:
        print("Train Merge models took: %.3f secs" % (time.time()-start))
    else:
        print("Valid Merge models took: %.3f secs" % (time.time()-start))


input_cols_1 = [
    (['fold', 'language'], "lookup_table_fold_language"),
    (['fold', 'tweet_dow'], "lookup_table_fold_tweet_dow"),
    (['fold', 'tweet_type'], "lookup_table_fold_tweet_type"),
    (['fold', 'engaged_with_user_id'], "lookup_table_engaged_with_user_id_fold"),
    (['fold', 'engaging_user_id'], "lookup_table_fold_engaging_user_id"),
]

input_cols_2 = [
    (['fold', 'engaging_user_id', 'engaged_with_user_id'], "lookup_table_engaged_with_user_id_fold_engaging_user_id"),
    (['fold', 'engaging_user_id', 'language'], "lookup_table_fold_language_engaging_user_id"),
    (['fold', 'engaging_user_id', 'tweet_dow'], "lookup_table_fold_tweet_dow_engaging_user_id"),
    (['fold', 'engaging_user_id', 'tweet_hour'], "lookup_table_fold_engaging_user_id_tweet_hour"), 
    
]

input_cols_3 = [    
    (['fold', 'engaged_with_user_id', 'tweet_dow'], "lookup_table_engaged_with_user_id_fold_tweet_dow"),
    (['fold', 'engaged_with_user_id', 'tweet_hour'], "lookup_table_engaged_with_user_id_fold_tweet_hour"),
    (['fold', 'engaged_with_user_id', 'tweet_type'], "lookup_table_engaged_with_user_id_fold_tweet_type"),
]

input_cols_4 = [
    (['fold', 'engaging_user_id', 'tweet_type'], "lookup_table_fold_tweet_type_engaging_user_id"),
    (['fold', 'engaged_with_user_id', 'has_rt'], "lookup_table_engaged_with_user_id_fold_has_rt"),
    (['fold', 'engaged_with_user_id', 'language'], "lookup_table_engaged_with_user_id_fold_language"),
]


############## Train join #############
train_out_dfs = []
proc = DataProcessor(spark, path_prefix,
                     current_path=current_path, dicts_path=dicts_folder)

#train_res_dfs = {}
#for in_cols, dict_name in input_cols_1:
#    train_res_dfs[dict_name] = (in_cols, spark.read.parquet("%s/train_lookup/%s" % (current_path, dict_name)), 0)
#train = spark.read.parquet("%s/train_with_features_has_rt" % current_path)
#merge_indicator_by_input_cols(proc, train, train_res_dfs, is_train=True, out_name = "train_with_features_1")

#train_res_dfs = {}
#for in_cols, dict_name in input_cols_2:
#    train_res_dfs[dict_name] = (in_cols, spark.read.parquet("%s/train_lookup/%s" % (current_path, dict_name)), 0)
#train = spark.read.parquet("%s/train_with_features_1" % current_path)
#merge_indicator_by_input_cols(proc, train, train_res_dfs, is_train=True, out_name = "train_with_features_2")
#
#train_res_dfs = {}
#for in_cols, dict_name in input_cols_3:
#    train_res_dfs[dict_name] = (in_cols, spark.read.parquet("%s/train_lookup/%s" % (current_path, dict_name)), 0)    
#train = spark.read.parquet("%s/train_with_features_2" % current_path)
#merge_indicator_by_input_cols(proc, train, train_res_dfs, is_train=True, out_name = "train_with_features_3")

train_res_dfs = {}
for in_cols, dict_name in input_cols_4:
    train_res_dfs[dict_name] = (in_cols, spark.read.parquet("%s/train_lookup/%s" % (current_path, dict_name)), 0)    
train = spark.read.parquet("%s/train_with_features_3" % current_path)
merge_indicator_by_input_cols(proc, train, train_res_dfs, is_train=True, out_name = "train_with_features")

Train Merge models took: 1814.322 secs


In [9]:
df = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features")
df.write.format('parquet').mode('overwrite').save("file:///mnt/nvme2/chendi/BlueWhale/sample_0_3/stage4_20days/train_with_features")

In [7]:
valid = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days/validate_with_features")
valid = valid.withColumn('tweet_dow', F.dayofweek(F.from_unixtime(F.col('tweet_timestamp'))).cast(IntegerType()))\
             .withColumn('tweet_hour', F.hour(F.from_unixtime(F.col('tweet_timestamp'))).cast(IntegerType()))\
             .withColumn('has_rt', F.col('tweet').rlike("^RT\s+|\sRT\s+"))\
             .withColumn('dt_dow', F.dayofweek(F.from_unixtime(F.col('tweet_timestamp'))).cast(IntegerType()))\
             .withColumn('dt_hour', F.hour(F.from_unixtime(F.col('tweet_timestamp'))).cast(IntegerType()))\
             .withColumn('dt_minute', F.minute(F.from_unixtime(F.col('tweet_timestamp'))).cast(IntegerType()))\
             .withColumn('dt_second', F.second(F.from_unixtime(F.col('tweet_timestamp'))).cast(IntegerType()))\
             .withColumn('a_ff_rate', F.col("engaged_with_user_following_count")/F.col("engaged_with_user_follower_count"))\
             .withColumn('b_ff_rate', F.col("engaging_user_following_count") /F.col("engaging_user_follower_count"))
valid = valid.fillna(0, ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp'])
valid = valid.withColumn('reply_timestamp', F.when(F.col('reply_timestamp') > 0, 1).otherwise(0))\
             .withColumn('retweet_timestamp', F.when(F.col('retweet_timestamp') > 0, 1).otherwise(0))\
             .withColumn('retweet_with_comment_timestamp', F.when(F.col('retweet_with_comment_timestamp') > 0, 1).otherwise(0))\
             .withColumn('like_timestamp', F.when(F.col('like_timestamp') > 0, 1).otherwise(0))

def merge_indicator_by_input_cols(proc, df, te, is_train=True, out_name="validate_with_features"):
    start = time.time()
    op_merge = ModelMerge(te)
    proc.reset_ops([op_merge])
    proc.transform(df, name=out_name)
    if is_train:
        print("Train Merge models took: %.3f secs" % (time.time()-start))
    else:
        print("Valid Merge models took: %.3f secs" % (time.time()-start))


input_cols_1 = [
    (['language'], "lookup_table_fold_language"),
    (['tweet_dow'], "lookup_table_fold_tweet_dow"),
    (['tweet_type'], "lookup_table_fold_tweet_type"),
    (['engaged_with_user_id'], "lookup_table_engaged_with_user_id_fold"),
    (['engaging_user_id'], "lookup_table_fold_engaging_user_id"),
    (['engaging_user_id', 'engaged_with_user_id'], "lookup_table_engaged_with_user_id_fold_engaging_user_id"),
    (['engaging_user_id', 'language'], "lookup_table_fold_language_engaging_user_id"),
    (['engaging_user_id', 'tweet_dow'], "lookup_table_fold_tweet_dow_engaging_user_id"),
    (['engaging_user_id', 'tweet_hour'], "lookup_table_fold_engaging_user_id_tweet_hour"), 
    (['engaged_with_user_id', 'tweet_dow'], "lookup_table_engaged_with_user_id_fold_tweet_dow"),
    (['engaged_with_user_id', 'tweet_hour'], "lookup_table_engaged_with_user_id_fold_tweet_hour"),
    (['engaged_with_user_id', 'tweet_type'], "lookup_table_engaged_with_user_id_fold_tweet_type"),
    (['engaging_user_id', 'tweet_type'], "lookup_table_fold_tweet_type_engaging_user_id"),
    (['engaged_with_user_id', 'has_rt'], "lookup_table_engaged_with_user_id_fold_has_rt"),
    (['engaged_with_user_id', 'language'], "lookup_table_engaged_with_user_id_fold_language"),
]


############## Train join #############
current_path = "/recsys2021_0608_processed/sample_0_3_20days_eric_features"
valid_out_dfs = []
proc = DataProcessor(spark, path_prefix, current_path=current_path, dicts_path=dicts_folder)

valid_res_dfs = {}
for in_cols, dict_name in input_cols_1:
    valid_res_dfs[dict_name] = (in_cols, spark.read.parquet("%s/valid_lookup/%s" % (current_path, dict_name)), 0)
merge_indicator_by_input_cols(proc, valid, valid_res_dfs, is_train=False, out_name = "validate_with_features")


Valid Merge models took: 176.348 secs


In [11]:
valid = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/validate_with_features")
train = spark.read.parquet("/recsys2021_0608_processed/sample_0_3_20days_eric_features/train_with_features")

train_schema = train.columns
valid_schema = valid.columns

for c in train_schema:
    if c not in valid_schema:
        print("%s is not in validate" % c)
valid.printSchema()

fold is not in validate
categorified_hashtags is not in validate
categorified_tweet_id is not in validate
categorified_present_media is not in validate
categorified_present_links is not in validate
categorified_present_domains is not in validate
categorified_tweet_type is not in validate
categorified_language is not in validate
categorified_engaged_with_user_id is not in validate
categorified_engaging_user_id is not in validate
tokens is not in validate
len_hashtags is not in validate
len_domains is not in validate
len_links is not in validate
engage_time is not in validate
tweet_nortsign is not in validate
count_words is not in validate
count_char is not in validate
tw_uhash is not in validate
tw_hash is not in validate
count_ats is not in validate
tw_hash0 is not in validate
tw_hash1 is not in validate
tw_second_word is not in validate
tw_llast_word is not in validate
tw_len is not in validate
original_tweet is not in validate
elapsed_time is not in validate
original_tw_first_word is

In [23]:
our_feature_list = [
"TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_reply_timestamp",
"TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_timestamp",
"TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_with_comment_timestamp",
"TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_like_timestamp",
"TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_reply_timestamp",
"TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_timestamp",
"TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_with_comment_timestamp",
"TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_like_timestamp",
"TE_present_domains_present_media_tweet_type_language_reply_timestamp",
"TE_present_domains_present_media_tweet_type_language_like_timestamp",
"TE_present_links_present_media_tweet_type_language_reply_timestamp",
"TE_hashtags_present_media_tweet_type_language_reply_timestamp",
"TE_hashtags_present_media_tweet_type_language_retweet_timestamp",
"TE_hashtags_present_media_tweet_type_language_retweet_with_comment_timestamp",
"TE_hashtags_present_media_tweet_type_language_like_timestamp",
"TE_engaged_with_user_id_reply_timestamp",
"TE_engaged_with_user_id_retweet_timestamp",
"TE_engaged_with_user_id_retweet_with_comment_timestamp",
"TE_engaged_with_user_id_like_timestamp",
"CE_engaged_with_user_id",
"CE_engaged_with_user_id_norm",
"TE_engaging_user_id_reply_timestamp",
"TE_engaging_user_id_retweet_timestamp",
"TE_engaging_user_id_retweet_with_comment_timestamp",
"TE_engaging_user_id_like_timestamp",
"CE_engaging_user_id",
"CE_engaging_user_id_norm",
"TE_engaged_with_user_id_tweet_type_language_reply_timestamp",
"TE_engaged_with_user_id_tweet_type_language_retweet_timestamp",
"TE_engaged_with_user_id_tweet_type_language_retweet_with_comment_timestamp",
"TE_engaged_with_user_id_tweet_type_language_like_timestamp",
"TE_tw_hash0_tweet_type_language_reply_timestamp",
"TE_tw_hash0_tweet_type_language_retweet_timestamp",
"TE_tw_hash0_tweet_type_language_retweet_with_comment_timestamp",
"TE_tw_hash0_tweet_type_language_like_timestamp",
"TE_tw_hash1_tweet_type_language_retweet_with_comment_timestamp",
"TE_tw_uhash_tweet_type_language_retweet_with_comment_timestamp",
"dt_dow",
"dt_hour",
"dt_minute",
"dt_second",
"a_ff_rate",
"b_ff_rate",
]

eric_feature_list = feature_list_after_merge

overlap = [
"TE_engaged_with_user_id_reply_timestamp",
"TE_engaged_with_user_id_reply_timestamp",
"TE_engaged_with_user_id_retweet_timestamp",
"TE_engaged_with_user_id_retweet_timestamp",
"TE_engaged_with_user_id_retweet_with_comment_timestamp",
"TE_engaged_with_user_id_retweet_with_comment_timestamp",
"TE_engaged_with_user_id_like_timestamp",
"TE_engaged_with_user_id_like_timestamp",
"TE_engaging_user_id_reply_timestamp",
"TE_engaging_user_id_retweet_timestamp",
"TE_engaging_user_id_retweet_with_comment_timestamp",
"TE_engaging_user_id_like_timestamp",
]

In [25]:
numFeatures = 0
for c in our_feature_list:
    if c not in overlap:
        print(c)
        numFeatures += 1
print(numFeatures)


TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_reply_timestamp
TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_timestamp
TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_with_comment_timestamp
TE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_like_timestamp
TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_reply_timestamp
TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_timestamp
TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_with_comment_timestamp
TE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_

In [21]:
for c in our_feature_list:
    for ec in eric_feature_list:
        if c in ec:
            print(c)

TE_engaged_with_user_id_reply_timestamp
TE_engaged_with_user_id_reply_timestamp
TE_engaged_with_user_id_retweet_timestamp
TE_engaged_with_user_id_retweet_timestamp
TE_engaged_with_user_id_retweet_with_comment_timestamp
TE_engaged_with_user_id_retweet_with_comment_timestamp
TE_engaged_with_user_id_like_timestamp
TE_engaged_with_user_id_like_timestamp
TE_engaging_user_id_reply_timestamp
TE_engaging_user_id_retweet_timestamp
TE_engaging_user_id_retweet_with_comment_timestamp
TE_engaging_user_id_like_timestamp


In [15]:
good_features = {"like_timestamp_indicator": [
	"has_rt", "engaged_with_user_follower_count", "engaged_with_user_is_verified", "engaging_user_follower_count", "engaging_user_following_count", "engagee_follows_engager", "has_photo", "has_video", "has_gif", "has_links", "has_hashtags", "engaged_user_rate", "engaging_user_rate", "same_languages", "TE_engaging_user_id_reply_timestamp_indicator", "TE_engaging_user_id_retweet_timestamp_indicator", "TE_engaging_user_id_retweet_with_comment_timestamp_indicator", "TE_engaging_user_id_like_timestamp_indicator", "TE_language_retweet_timestamp_indicator", "TE_language_retweet_with_comment_timestamp_indicator", "TE_language_like_timestamp_indicator", "TE_tweet_type_reply_timestamp_indicator", "TE_tweet_type_retweet_timestamp_indicator", "TE_tweet_type_like_timestamp_indicator", "TE_engaged_with_user_id_reply_timestamp_indicator", "TE_engaged_with_user_id_retweet_timestamp_indicator", "TE_engaged_with_user_id_retweet_with_comment_timestamp_indicator", "TE_engaged_with_user_id_like_timestamp_indicator", "TE_tweet_dow_retweet_timestamp_indicator", "GTE_language_reply_timestamp_indicator_engaging_user_id", "GTE_language_retweet_timestamp_indicator_engaging_user_id", "GTE_language_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_language_like_timestamp_indicator_engaging_user_id", "GTE_tweet_type_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_type_retweet_timestamp_indicator_engaging_user_id", "GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_type_like_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_retweet_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_like_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_retweet_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_like_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_reply_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_retweet_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_like_timestamp_indicator_engaging_user_id", "GTE_has_rt_reply_timestamp_indicator_engaged_with_user_id", "GTE_has_rt_retweet_timestamp_indicator_engaged_with_user_id", "GTE_has_rt_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_has_rt_like_timestamp_indicator_engaged_with_user_id", "GTE_language_retweet_timestamp_indicator_engaged_with_user_id", "GTE_language_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_language_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_retweet_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_retweet_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_retweet_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_like_timestamp_indicator_engaged_with_user_id"
], 
"retweet_with_comment_timestamp_indicator": [
	"has_gif", "TE_engaging_user_id_retweet_with_comment_timestamp_indicator", "TE_engaged_with_user_id_retweet_with_comment_timestamp_indicator", "GTE_language_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_has_rt_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_language_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaged_with_user_id"
], 
"reply_timestamp_indicator": [
	"engaged_with_user_follower_count", "engaging_user_follower_count", "engagee_follows_engager", "has_links", "TE_engaging_user_id_reply_timestamp_indicator", "TE_engaging_user_id_like_timestamp_indicator", "TE_language_retweet_timestamp_indicator", "TE_tweet_type_reply_timestamp_indicator", "TE_engaged_with_user_id_reply_timestamp_indicator", "GTE_language_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_type_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_reply_timestamp_indicator_engaging_user_id", "GTE_has_rt_reply_timestamp_indicator_engaged_with_user_id", "GTE_language_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaged_with_user_id"
], 
"retweet_timestamp_indicator": [
	"has_rt", "engaged_with_user_follower_count", "engaged_with_user_is_verified", "engaging_user_follower_count", "engagee_follows_engager", "has_video", "has_gif", "engaged_user_rate", "same_languages", "TE_engaging_user_id_reply_timestamp_indicator", "TE_engaging_user_id_retweet_timestamp_indicator", "TE_engaging_user_id_retweet_with_comment_timestamp_indicator", "TE_engaging_user_id_like_timestamp_indicator", "TE_language_reply_timestamp_indicator", "TE_language_retweet_timestamp_indicator", "TE_language_retweet_with_comment_timestamp_indicator", "TE_language_like_timestamp_indicator", "TE_tweet_type_retweet_timestamp_indicator", "TE_engaged_with_user_id_reply_timestamp_indicator", "TE_engaged_with_user_id_retweet_timestamp_indicator", "TE_engaged_with_user_id_retweet_with_comment_timestamp_indicator", "TE_engaged_with_user_id_like_timestamp_indicator", "GTE_language_reply_timestamp_indicator_engaging_user_id", "GTE_language_retweet_timestamp_indicator_engaging_user_id", "GTE_language_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_language_like_timestamp_indicator_engaging_user_id", "GTE_tweet_type_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_type_retweet_timestamp_indicator_engaging_user_id", "GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_type_like_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_retweet_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_hour_like_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_reply_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_retweet_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_tweet_dow_like_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_reply_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_retweet_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_retweet_with_comment_timestamp_indicator_engaging_user_id", "GTE_engaged_with_user_id_like_timestamp_indicator_engaging_user_id", "GTE_has_rt_retweet_timestamp_indicator_engaged_with_user_id", "GTE_has_rt_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_has_rt_like_timestamp_indicator_engaged_with_user_id", "GTE_language_reply_timestamp_indicator_engaged_with_user_id", "GTE_language_retweet_timestamp_indicator_engaged_with_user_id", "GTE_language_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_language_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_retweet_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_type_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_retweet_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_hour_like_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_reply_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_retweet_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_retweet_with_comment_timestamp_indicator_engaged_with_user_id", "GTE_tweet_dow_like_timestamp_indicator_engaged_with_user_id"
]}
good_features_list = [f for fl in good_features.values() for f in fl ]

In [26]:
#print(good_features_list)
len(good_features_list)
for c in good_features_list:
    print(c)

has_rt
engaged_with_user_follower_count
engaged_with_user_is_verified
engaging_user_follower_count
engaging_user_following_count
engagee_follows_engager
has_photo
has_video
has_gif
has_links
has_hashtags
engaged_user_rate
engaging_user_rate
same_languages
TE_engaging_user_id_reply_timestamp_indicator
TE_engaging_user_id_retweet_timestamp_indicator
TE_engaging_user_id_retweet_with_comment_timestamp_indicator
TE_engaging_user_id_like_timestamp_indicator
TE_language_retweet_timestamp_indicator
TE_language_retweet_with_comment_timestamp_indicator
TE_language_like_timestamp_indicator
TE_tweet_type_reply_timestamp_indicator
TE_tweet_type_retweet_timestamp_indicator
TE_tweet_type_like_timestamp_indicator
TE_engaged_with_user_id_reply_timestamp_indicator
TE_engaged_with_user_id_retweet_timestamp_indicator
TE_engaged_with_user_id_retweet_with_comment_timestamp_indicator
TE_engaged_with_user_id_like_timestamp_indicator
TE_tweet_dow_retweet_timestamp_indicator
GTE_language_reply_timestamp_indicat